# Exercise 1

a) No, we do not observe equal values for maximal activities along a linear pathway, while fluxes are forced to be equal in steady state due to mass balance constraints, maximal activities are instrinsic enzyme properties and this differs across different reactions, this is why reaction in the same linear pathway display varying activities rather than identical ones.
b) EX_gln__L_e,EX_pyr_e
	Exchange reactions dont represent enzymatic steps inside the cell but rather material flow across the system boundary. Maximal activity is derived from enzyme expression, exchange reactions are not catalysed by specific intracellular enzymes, therefore, there is no associated gene to the reaction and thus, no maximal activity.


# Exercise 2

In [44]:
import cobra
from cobra.io import load_json_model
import pandas as pd

In [36]:

# Load model
model = load_json_model("e_coli_core.json")

max_activity = pd.read_csv("e_coli_core_expression.csv")
max_activity.columns = [col.strip() for col in max_activity.columns]

max_activity_dict = dict(zip(max_activity["# Reaction ID"], max_act["reaction activity [mmol/gDW/h]"]))

for reaction_key in max_act_dict:
    # try:
        reaction = model.reactions.get_by_id(reaction_key)
        value = max_act_dict[reaction_key]
    # Change reversible reactions bounds
        if reaction.reversibility:
            reaction.upper_bound = value
            reaction.lower_bound = -value
    # Change irreversible reaction bounds
        else:
            reaction.upper_bound = value
            reaction.lower_bound = 0
    # except KeyError:
    #     pass
             
        

        
    
        

In [43]:
# glucose exhange reaction
glucose_ex = model.reactions.get_by_id("EX_glu__L_e")
print(glucose_ex.upper_bound)
print(glucose_ex.lower_bound)

# We just need to change lower bound as upper bound is already 1000

glucose_ex.lower_bound = -1000


# ATPM 
atpm = model.reactions.get_by_id("ATPM")

for reac in model.reactions:
    print(f"{reac.id} uppper bound: {reac.upper_bound}, lower bound: {reac.lower_bound}")
    
                       

1000.0
-1000
PFK uppper bound: 12.12, lower bound: 0
PFL uppper bound: 1.0, lower bound: 0
PGI uppper bound: 13.12, lower bound: -13.12
PGK uppper bound: 23.13, lower bound: -23.13
PGL uppper bound: 8.12, lower bound: 0
ACALD uppper bound: 1.16, lower bound: -1.16
AKGt2r uppper bound: 3.1, lower bound: -3.1
PGM uppper bound: 20.01, lower bound: -20.01
PIt2r uppper bound: 6.03, lower bound: -6.03
ALCD2x uppper bound: 9.01, lower bound: -9.01
ACALDt uppper bound: 2.29, lower bound: -2.29
ACKr uppper bound: 1.19, lower bound: -1.19
PPC uppper bound: 2.56, lower bound: 0
ACONTa uppper bound: 25.35, lower bound: -25.35
ACONTb uppper bound: 25.35, lower bound: -25.35
ATPM uppper bound: 1000.0, lower bound: 8.39
PPCK uppper bound: 25.23, lower bound: 0
ACt2r uppper bound: 3.23, lower bound: -3.23
PPS uppper bound: 2.5, lower bound: 0
ADK1 uppper bound: 30.57, lower bound: -30.57
AKGDH uppper bound: 24.35, lower bound: 0
ATPS4r uppper bound: 60.5, lower bound: -60.5
PTAr uppper bound: 4.47, lo

# Exercise 3

### a)

In [45]:
from cobra.flux_analysis import flux_variability_analysis

fva_result = flux_variability_analysis(model, reaction_list=model.reactions)
print("Reaction ID\tMin Flux\tMax Flux")
for rxn_id, row in fva_result.iterrows():
    print(f"{rxn_id}\t{row['minimum']}\t{row['maximum']}")

Reaction ID	Min Flux	Max Flux
PFK	4.87817213515311	4.878172135152738
PFL	3.453562173728181e-13	0.0
PGI	5.29157178464768	5.291571784646887
PGK	-9.406182151087208	-9.406182151087776
PGL	0.0	-6.92440013346954e-13
ACALD	-1.16	-1.1600000000000965
AKGt2r	1.875941920547251e-13	0.0
PGM	-8.623041875484747	-8.623041875485313
PIt2r	1.9257607833280492	1.92576078332804
ALCD2x	-1.16	-1.1600000000001438
ACALDt	1.415413863569169e-13	-2.1131382567667543e-14
ACKr	-1.19	-1.1900000000001527
PPC	0.0	-2.4942245635375176e-13
ACONTa	1.8463120703642544	1.8463120703638862
ACONTb	1.8463120703642164	1.8463120703638847
ATPM	8.39	8.389999999999878
PPCK	0.0	-2.769760053387808e-13
ACt2r	-1.19	-1.1900000000001507
PPS	0.0	-1.1916850692457032e-13
ADK1	0.0	-1.2880833507502013e-13
AKGDH	5.401519260639961	5.401519260639652
ATPS4r	25.76182197337541	25.761821973374744
PTAr	1.1900000000001523	1.19
PYK	2.952411351825075	2.952411351824594
BIOMASS_Ecoli_core_w_GAM	0.523489489039076	0.5234894890390742
PYRt2	-1.26	-1.2600000000003

### b)

In [48]:
constrained_reactions = []

for reaction in model.reactions:
    if reaction.id == "FORt":
        continue
    max_flux = fva_result.loc[reaction.id, "maximum"]
    if max_flux >0 and max_flux < reaction.upper_bound:
        constrained_reactions.append(reaction.id)

print(f"Number of reactions with constrained maximal flux: {len(constrained_reactions)}")
print("Reactions:", constrained_reactions)

        

        

Number of reactions with constrained maximal flux: 36
Reactions: ['PFK', 'PGI', 'PIt2r', 'ACONTa', 'ACONTb', 'ATPM', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'BIOMASS_Ecoli_core_w_GAM', 'CS', 'CYTBD', 'ENO', 'THD2', 'TPI', 'EX_ac_e', 'EX_co2_e', 'EX_etoh_e', 'EX_h_e', 'EX_h2o_e', 'EX_lac__D_e', 'EX_nh4_e', 'EX_pyr_e', 'FBA', 'FRD7', 'FUM', 'GAPD', 'GLCpts', 'GLNS', 'GLUDy', 'ICDHyr', 'MDH', 'ME2', 'O2t', 'PDH']


Reactions behave this way due to certain network and regulatory constraints, even though the reaction could theoretically go up to its reaction upper bound, FVA respects all constraints simoultaneously, serving as a bottleneck to the flux.
Thus, the reaction is limited by the network contraints.

### c)

In [50]:
pos_min_flux_reac = []
for reaction_id, row in fva_result.iterrows():
    if row['minimum'] >0:
        pos_min_flux_reac.append(reaction_id)
        
print(f"Number of reactions with positive minimal flux: {len(pos_min_flux_reac)}")
print("Reactions:", pos_min_flux_reac)

Number of reactions with positive minimal flux: 42
Reactions: ['PFK', 'PFL', 'PGI', 'AKGt2r', 'PIt2r', 'ACALDt', 'ACONTa', 'ACONTb', 'ATPM', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'BIOMASS_Ecoli_core_w_GAM', 'CS', 'CYTBD', 'ENO', 'SUCDi', 'THD2', 'TPI', 'EX_ac_e', 'EX_co2_e', 'EX_etoh_e', 'EX_h_e', 'EX_h2o_e', 'EX_lac__D_e', 'EX_nh4_e', 'EX_pyr_e', 'FBA', 'FUM', 'GAPD', 'GLCpts', 'GLNS', 'GLNabc', 'GLUDy', 'GLUt2r', 'ICDHyr', 'MDH', 'ME2', 'NADH16', 'O2t', 'PDH']


Reactions behave this way due to"
- to satisfy stoichiometry,  mass balance or biomass production
- Part of a pathway where flux must flow continously
- Gene exression or transport constraint may also enforce minimal flux directly

These reactions essentially are always on the feasible solution space, so the network requires them to alsway carry some flux,

# Exercise 4

### a) 

In [51]:
for reaction in model.reactions:
    if "biomass" in rxn.id.lower():
        print(rxn.id)

BIOMASS_Ecoli_core_w_GAM


In [54]:
model.objective = model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM")

solution = model.optimize()

print(f"Maximal biomass production rate: {solution.objective_value}")




Maximal biomass production rate: 0.5234894890390746


### b)

In [58]:
bottlenecks = []

for reaction in model.reactions:
    flux = solution.fluxes[reaction.id]
    if flux == reaction.upper_bound: # reaction reached its maximum flux
        bottlenecks.append(reaction.id)
print(f"Number of bottleneck reactions: {len(bottlenecks)}")
print("Bottleneck reactions:", bottlenecks)


Number of bottleneck reactions: 2
Bottleneck reactions: ['GLUt2r', 'NADH16']


### c)

In [62]:
maximal_constrained_reactions = []
for reaction in model.reactions:
    if reaction.upper_bound < 1000:
        maximal_constrained_reactions.append(reaction)
print(f"Reactions with maximal activity constraints: {[reaction.id for reaction in maximal_constrained_reactions]}")

zero_flux_reactions = []
for reaction in maximal_constrained_reactions:
    if solution.fluxes[reaction.id] == 0:
        zero_flux_reactions.append(reaction)

print(f"Example reaction with maximal activity but zero flux: {zero_flux_reactions[0].id}")

        
        
    


Reactions with maximal activity constraints: ['PFK', 'PFL', 'PGI', 'PGK', 'PGL', 'ACALD', 'AKGt2r', 'PGM', 'PIt2r', 'ALCD2x', 'ACALDt', 'ACKr', 'PPC', 'ACONTa', 'ACONTb', 'PPCK', 'ACt2r', 'PPS', 'ADK1', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'PYRt2', 'CO2t', 'RPE', 'CS', 'RPI', 'SUCCt2_2', 'CYTBD', 'D_LACt2', 'ENO', 'SUCCt3', 'ETOHt2r', 'SUCDi', 'SUCOAS', 'TALA', 'THD2', 'TKT1', 'TKT2', 'TPI', 'FBA', 'FBP', 'FORt2', 'FORt', 'FRD7', 'FRUpts2', 'FUM', 'FUMt2_2', 'G6PDH2r', 'GAPD', 'GLCpts', 'GLNS', 'GLNabc', 'GLUDy', 'GLUN', 'GLUSy', 'GLUt2r', 'GND', 'H2Ot', 'ICDHyr', 'ICL', 'LDH_D', 'MALS', 'MALt2_2', 'MDH', 'ME1', 'ME2', 'NADH16', 'NADTRHD', 'NH4t', 'O2t', 'PDH']
Example reaction with maximal activity but zero flux: PFL


PFL reaction converts pyruvate to formate and acetyl-CoA.
Pyruvate can also be converted to acetyl-CoA via PDH or other reactions, and under maximal biomass optimisation, the network may choose other pathway that maximise growth, ignoring PFL.
PFL is anaerobic, so the network may prefer PDH under aeorbic conditions.
In summary, PFL has  maximal flux constraint, but in the FBA solution for maximal biomass, its flux is zero, this can be because the network can convert pyruvate to acetyl-CoA via PDH instad, which  is a more favorable alterative for biomass maximisation